In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Benin Solar Data Analysis\n",
    "\n",
    "## Data Loading and Cleaning"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {},
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from scipy import stats\n",
    "\n",
    "# Load data\n",
    "df = pd.read_csv('../data/benin-malanville.csv')\n",
    "\n",
    "# Basic cleaning and analysis steps here\n",
    "# Save cleaned data\n",
    "df.to_csv('../data/benin_clean.csv', index=False)"
   ]
  }
 ]
}